In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
% config InlineBackend.figure_format = 'png'
% matplotlib inline

In [41]:
data = pd.read_csv("census/census-income.data")

In [44]:
categorical_attributes = ['ACLSWKR','ADTIND','ADTOCC','AHGA','AHSCOL','AMARITL','AMJIND','AMJOCC','ARACE',
                          'AREORGN','ASEX','AUNMEM','AUNTYPE','AWKSTAT','FILESTAT','GRINREG','GRINST','HHDFMX',
                          'HHDREL','MIGMTR1','MIGMTR3','MIGMTR4','MIGSAME','MIGSUN','PARENT','PEFNTVTY',
                          'PEMNTVTY','PENATVTY','PRCITSHP','SEOTR','VETQVA','VETYN','YEAR']

In [176]:
data_categorical = data.loc[:,categorical_attributes]

In [203]:
# Fix missing values to mode
data_categorical.loc[data_categorical.GRINST == " ?", "GRINST"] = \
data_categorical.loc[data_categorical.GRINST != " ?", "GRINST"].mode().iloc[0]

data_categorical.loc[data_categorical.MIGMTR3 == " ?", "MIGMTR3"] = \
data_categorical.loc[data_categorical.MIGMTR3 != " ?", "MIGMTR3"].mode().iloc[0]

data_categorical.loc[data_categorical.MIGMTR3 == " ?", "MIGMTR3"] = \
data_categorical.loc[data_categorical.MIGMTR3 != " ?", "MIGMTR3"].mode().iloc[0]

data_categorical.loc[data_categorical.MIGMTR4 == " ?", "MIGMTR4"] = \
data_categorical.loc[data_categorical.MIGMTR4 != " ?", "MIGMTR4"].mode().iloc[0]

data_categorical.loc[data_categorical.MIGSAME == " ?", "MIGSAME"] = \
data_categorical.loc[data_categorical.MIGSAME != " ?", "MIGSAME"].mode().iloc[0]

data_categorical.loc[data_categorical.PEFNTVTY == " ?", "PEFNTVTY"] = \
data_categorical.loc[data_categorical.PEFNTVTY != " ?", "PEFNTVTY"].mode().iloc[0]

data_categorical.loc[data_categorical.PEMNTVTY == " ?", "PEMNTVTY"] = \
data_categorical.loc[data_categorical.PEMNTVTY != " ?", "PEMNTVTY"].mode().iloc[0]

data_categorical.loc[data_categorical.PENATVTY == " ?", "PENATVTY"] = \
data_categorical.loc[data_categorical.PENATVTY != " ?", "PENATVTY"].mode().iloc[0]

In [96]:
data_randomised = data_categorical.iloc[np.random.permutation(data_categorical.shape[0])]
train = data_randomised.iloc[:99762]
test = data_randomised.iloc[99762:]

(99761, 33)